# Random Surfer Algorithm (PageRank Algorithm) の紹介

In [1]:
## create the clean environment
import gc
import matplotlib.pyplot as plt

def clear_all():
    # Clears all the variables from the workspace
    gl = globals().copy()
    for var in gl:
        if var in clean_env_var: continue
        del globals()[var]
    # Garbage collection:
    gc.collect()

def close_plots():
  my_plots = plt.get_fignums()
  for j in my_plots:
    plt.close(plt.figure(j))

clean_env_var = dir()
clean_env_var.append('clean_env_var')

In [2]:
clear_all()

### Hardware

In [3]:
%%bash
system_profiler SPHardwareDataType | grep -E \
"Model Identifier"\|"Processor Name"\|"Processor Speed"\
\|"Number of Processors"\|"Memory:"

      Model Identifier: MacBookPro13,1
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 2 GHz
      Number of Processors: 1
      Memory: 16 GB


### Python

In [4]:
!python -V

Python 3.7.4


### Import

In [5]:
import numpy as np
import pprint

## PageRankの考え方

- 多くのページから参照されているページの質は高い
- 質の高いページから参照されているページは質が高い（Authority）


```
多くの良質なページからリンクされているページはやはり良質なページである
```

### 今回扱うweb pageの構造

<img src = "./fig/page_rank_sample.png">


### 今回の実装方針

1. Page遷移行列, G, を作る(ノード同士が結ばれている場合は1 else 0, 行から列への移動)
2. 転置した行列Gのそれぞれの列ベクトルの総和が1となるように規格化して「推移確率行列」をつくる
3. 初期ベクトルとして，すべての要素が同じ値で，足して1になるような長さ7の列ベクトルを用意する
4. 初期ベクトルに何度か(例えば10回，あるいは収束するまで)推移確率行列を掛ける．この操作は，行列の最大固有値に属する固有ベクトルを見つけることに相当する．
5. 得られたベクトルの各要素が対応するページの得点とみなせ，得点順にランクが高くなる．

In [6]:
G = np.array([[0, 1, 1, 1, 1, 0, 1],
              [1, 0, 0, 0, 0, 0, 0],
              [1, 1, 0, 0, 0, 0, 0],
              [0, 1, 1, 0, 1, 0, 0],
              [1, 0, 1, 1, 0, 1, 0],
              [1, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0, 0]]
            )
G

array([[0, 1, 1, 1, 1, 0, 1],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0]])

In [7]:
t_G = G.T
t_G

array([[0, 1, 1, 0, 1, 1, 0],
       [1, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0]])

In [8]:
prob = 1/t_G.sum(0)
prob = np.diag(prob)
prob

array([[0.2       , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.5       , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.33333333, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.25      ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.5       , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        ]])

In [9]:
tran_mat = t_G @ prob
np.set_printoptions(precision=4)
print(tran_mat)

[[0.     1.     0.5    0.     0.25   0.5    0.    ]
 [0.2    0.     0.5    0.3333 0.     0.     0.    ]
 [0.2    0.     0.     0.3333 0.25   0.     0.    ]
 [0.2    0.     0.     0.     0.25   0.     0.    ]
 [0.2    0.     0.     0.3333 0.     0.5    1.    ]
 [0.     0.     0.     0.     0.25   0.     0.    ]
 [0.2    0.     0.     0.     0.     0.     0.    ]]


In [10]:
init = np.ones(len(G))/len(G)
init

array([0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429])

In [11]:
tran_mat @ tran_mat

array([[0.35  , 0.    , 0.5   , 0.5833, 0.25  , 0.125 , 0.25  ],
       [0.1667, 0.2   , 0.1   , 0.1667, 0.2583, 0.1   , 0.    ],
       [0.1167, 0.2   , 0.1   , 0.0833, 0.1333, 0.225 , 0.25  ],
       [0.05  , 0.2   , 0.1   , 0.0833, 0.05  , 0.225 , 0.25  ],
       [0.2667, 0.2   , 0.1   , 0.    , 0.2583, 0.1   , 0.    ],
       [0.05  , 0.    , 0.    , 0.0833, 0.    , 0.125 , 0.25  ],
       [0.    , 0.2   , 0.1   , 0.    , 0.05  , 0.1   , 0.    ]])

In [12]:
np.linalg.matrix_power(tran_mat, 2)

array([[0.35  , 0.    , 0.5   , 0.5833, 0.25  , 0.125 , 0.25  ],
       [0.1667, 0.2   , 0.1   , 0.1667, 0.2583, 0.1   , 0.    ],
       [0.1167, 0.2   , 0.1   , 0.0833, 0.1333, 0.225 , 0.25  ],
       [0.05  , 0.2   , 0.1   , 0.0833, 0.05  , 0.225 , 0.25  ],
       [0.2667, 0.2   , 0.1   , 0.    , 0.2583, 0.1   , 0.    ],
       [0.05  , 0.    , 0.    , 0.0833, 0.    , 0.125 , 0.25  ],
       [0.    , 0.2   , 0.1   , 0.    , 0.05  , 0.1   , 0.    ]])

#### simulation 10回

In [13]:
np.linalg.matrix_power(tran_mat, 10) @ init

array([0.3033, 0.1663, 0.1406, 0.1053, 0.1792, 0.0446, 0.0607])

#### simulation 1000回

In [14]:
np.linalg.matrix_power(tran_mat, 1000) @ init

array([0.3035, 0.1661, 0.1406, 0.1054, 0.1789, 0.0447, 0.0607])

#### 初期ベクトルをランダムに選ぶ

In [15]:
init_2 = np.random.randint(1, 10, 7)
init_2 = init_2/init_2.sum()
init_2

array([0.1892, 0.1892, 0.1351, 0.0811, 0.2432, 0.027 , 0.1351])

In [16]:
res = np.linalg.matrix_power(tran_mat, 100) @ init_2
res

array([0.3035, 0.1661, 0.1406, 0.1054, 0.1789, 0.0447, 0.0607])

### 固有ベクトルとの関連

In [17]:
# w: The eigenvalues, v: The eigenvectors
w, v = np.linalg.eig(tran_mat)

In [18]:
index = np.argmax(w)
eig = v.T[index]
eig

array([0.6995+0.j, 0.3829+0.j, 0.324 +0.j, 0.243 +0.j, 0.4123+0.j,
       0.1031+0.j, 0.1399+0.j])

In [19]:
eig * np.linalg.norm(res)/np.linalg.norm(eig)

array([0.3035+0.j, 0.1661+0.j, 0.1406+0.j, 0.1054+0.j, 0.1789+0.j,
       0.0447+0.j, 0.0607+0.j])

`numpy.linalg.eig`で出力されるeigenvectorはnormalizedされているので一致はしないが、順序は同じ。ノルムをコントロールすると一致する。

## 離脱を踏まえたPagerank
### web page構造

web pageがA, B, C, D, Eと5つ存在し、参照リンクは以下

- A→B
- B→E
- C→A
- D→A
- E→A

どのページにも等確率で流入し、0.15の確率でsession離脱する仮定をおく → 

### 期待するoutput

```
A > B > E > C = D
```


In [20]:
G = np.array([[0, 0.85, 0, 0, 0, 0.15],
              [0, 0, 0, 0, 0.85, 0.15],
              [0.85, 0, 0, 0, 0, 0.15],
              [0.85, 0, 0, 0, 0, 0.15],
              [0.85, 0, 0, 0, 0, 0.15],
              [0.2, 0.2, 0.2, 0.2, 0.2, 0]], dtype = float
            )
G

array([[0.  , 0.85, 0.  , 0.  , 0.  , 0.15],
       [0.  , 0.  , 0.  , 0.  , 0.85, 0.15],
       [0.85, 0.  , 0.  , 0.  , 0.  , 0.15],
       [0.85, 0.  , 0.  , 0.  , 0.  , 0.15],
       [0.85, 0.  , 0.  , 0.  , 0.  , 0.15],
       [0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.  ]])

In [21]:
t_G = G.T
init = np.ones(len(G))/len(G)
np.linalg.matrix_power(t_G, 1000) @ init

array([0.2888, 0.2716, 0.0261, 0.0261, 0.2569, 0.1304])